In [1]:
import z3
from itertools import combinations, product

I = list(range(1, 10))
P = list(product(I, I))
X = {p: z3.Int(f"X{p[0]}{p[1]}") for p in P}
diff = lambda p, q: {abs(p[0] - q[0]), abs(p[1] - q[1])}

Classic = (
    [z3.And(1 <= X[p], X[p] <= 9) for p in P]
    + [z3.Distinct([X[i, j] for j in I]) for i in I]
    + [z3.Distinct([X[i, j] for i in I]) for j in I]
    + [
        z3.Distinct([X[i + di, j + dj] for (di, dj) in product(range(3), repeat=2)])
        for (i, j) in product(range(1, 10, 3), repeat=2)
    ]
)

AntiKnight = [X[p] != X[q] for p, q in combinations(P, 2) if diff(p, q) == {1, 2}]
AntiKing = [X[p] != X[q] for p, q in combinations(P, 2) if diff(p, q) == {1}]
OrthNotAdj = [
    z3.And(X[p] - X[q] != 1, X[p] - X[q] != -1)
    for p, q in combinations(P, 2)
    if diff(p, q) == {0, 1}
]

Given = lambda gs: [X[p] == k for p, k in gs.items()]


def solve(*cs):
    solver = z3.Solver()
    for c in cs:
        solver.add(c)

    assert solver.check() == z3.sat
    m = solver.model()

    template = "\n".join(
        [
            "╭───────┬───────┬───────╮",
            "| x x x | x x x | x x x |",
            "| x x x | x x x | x x x |",
            "| x x x | x x x | x x x |",
            "├───────┼───────┼───────┤",
            "| x x x | x x x | x x x |",
            "| x x x | x x x | x x x |",
            "| x x x | x x x | x x x |",
            "├───────┼───────┼───────┤",
            "| x x x | x x x | x x x |",
            "| x x x | x x x | x x x |",
            "| x x x | x x x | x x x |",
            "╰───────┴───────┴───────╯",
        ]
    )
    it = iter(P)
    print("".join(str(m.evaluate(X[next(it)])) if t == "x" else t for t in template))

In [2]:
# Miracle Sudoku (https://www.youtube.com/watch?v=yKf9aUIxdb4)
solve(
    Classic,
    AntiKnight,
    AntiKing,
    OrthNotAdj,
    Given({(5, 3): 1, (6, 7): 2}),
)

╭───────┬───────┬───────╮
| 4 8 3 | 7 2 6 | 1 5 9 |
| 7 2 6 | 1 5 9 | 4 8 3 |
| 1 5 9 | 4 8 3 | 7 2 6 |
├───────┼───────┼───────┤
| 8 3 7 | 2 6 1 | 5 9 4 |
| 2 6 1 | 5 9 4 | 8 3 7 |
| 5 9 4 | 8 3 7 | 2 6 1 |
├───────┼───────┼───────┤
| 3 7 2 | 6 1 5 | 9 4 8 |
| 6 1 5 | 9 4 8 | 3 7 2 |
| 9 4 8 | 3 7 2 | 6 1 5 |
╰───────┴───────┴───────╯
